In [1]:
import pandas
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.spatial


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:


def processText(column,n):#to remove stop words,to convert into lowercase and root word
    import re

    corpus = []
    for i in range(n):
        # review = re.sub('[^a-zA-Z]', ' ', column[i])
        review = re.sub(r'\[[0-9]*\]',' ',column[i])
        review = re.sub(r'\s+',' ',review)
        review = review.lower()
        text = re.sub(r'\d',' ',review)
        review = re.sub(r'\s+',' ',review)

        # review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        all_stopwords = stopwords.words('english')
        all_stopwords.remove('not')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus


In [5]:
def embed(input):
  return model(input)

In [6]:


df = pandas.read_csv('./'+'MSFT_combine.csv')


df=df.dropna()
df = df[~(df['datetime'] >= '2023-01-01')]#dropping rows of year=2023

df=df.reset_index(drop=True)


print(df.columns)

titleColumn=df['title']
summaryColumn=df['summary']
dateColumn=df['datetime']
titleColumn=list(titleColumn)
summaryColumn=list(summaryColumn)

corpus=processText(titleColumn,len(titleColumn))#preprocessing title column
df['title']=corpus

summaryCorpus=processText(summaryColumn,len(summaryColumn))#preprocessing summary column
df['summary']=summaryCorpus


df = df.astype({'title':'string'})
df = df.astype({'summary':'string'})

print(df.tail(5))

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')
       datetime       close        open         low        high      volume  \
969  2022-12-22  237.594894  241.259995  233.869995  241.990005  28651700.0   
970  2022-12-23  238.133545  236.110001  233.940002  238.869995  21207000.0   
971  2022-12-27  236.367981  238.699997  235.830002  238.929993  16688600.0   
972  2022-12-28  233.944031  236.889999  234.169998  239.720001  17457100.0   
973  2022-12-29  240.407837  235.649994  235.649994  241.919998  19770700.0   



In [7]:
df.shape

(974, 29)

In [8]:
titleColumn=df['title']
titleColumn[0]

# df=df.fillna(0)

# df=df.dropna()

'[\'author demand social network india scan pictur ... - mspoweruser\', \'microsoft surfac laptop 2 review: once-great laptop ... - pcworld\', \'cabinet approv set microsoft global data centr qatar - peninsula\', \'xbox live down: server statu latest microsoft confirm 0x87dd0006 sign issu - express\', \'myanalyt avail offic 365 microsoft 365 suit - neowin\', \'microsoft get approv open data center qatar - mspoweruser\', \'microsoft quietli updat window mix realiti ui ... - mspoweruser\', \'multipl microsoft patent point xbox one control improv - windowslatest\', \'microsoft graph: everyth develop need know - techgenix\', \'microsoft announc two new microsoft 365 secur ... - mspoweruser\', \'how creat form microsoft form adjust set - thewindowsclub\', "qatar\' cabinet approv microsoft azur data center - datacenterdynamics", \'which compani best code qualiti google, apple, microsoft amazon? - forbes\', \'how microsoft ceo satya nadella led company’ biggest turnaround 2018 - financi expre

In [9]:
summaryColumn=df['summary']
summaryColumn[0]

'[\'we put us$349.99/£329.99 surfac headphon test hands-on review.\\nent surfac headphones, microsoft announc surfac event last october.\\nnonetheless, surfac headphon feel pleasantli light, mostli plastic design help feel comfort worn.\\nth headphon current avail surfac gray, product manag event would say anyth whether microsoft plan releas surfac headphon colours.\\nhowever, refer surfac devices, also avail colour mention headphon black, help surfac headphon stand out.\', \'paradox microsoft team offer all-in-on mod solut across pc xbox one.\\nparadox interact microsoft announc surpris new joint project today, titl paradox mods.\\nbil “the first-ev open mod platform enabl distribut user-cr game modif (mods) across multipl platforms”, paradox mod free offer xbox one pc.\\nto get thing rolling, paradox mod alreadi avail part free updat paradox interactive’ survival-strategi game surviv mars.\\nparadox mod support come game later time.\', \'thi isn\\\'t first time player abl add modifi 

In [10]:
df.head(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
0,2019-02-20,102.832375,107.860001,106.290001,107.940002,21607700.0,1.0,0.898049,2.534889e+07,57.981282,...,52185000.0,-15773000.0,-36887000.0,38260000.0,286556000.0,184226000.0,102330000.0,168992000.0,['we put us$349.99/£329.99 surfac headphon tes...,['author demand social network india scan pict...
1,2019-02-21,105.001297,106.900002,106.870003,109.480003,29063200.0,1.0,1.880497,1.512891e+08,63.482724,...,52185000.0,-15773000.0,-36887000.0,38260000.0,286556000.0,184226000.0,102330000.0,168992000.0,['seattl seahawk star russel wilson alreadi ea...,['microsoft co-found paul allen taught nfl sta...
2,2019-02-22,106.498444,110.050003,109.820000,111.199997,27763200.0,1.0,3.052486,2.298640e+08,66.721646,...,52185000.0,-15773000.0,-36887000.0,38260000.0,286556000.0,184226000.0,102330000.0,168992000.0,['do idea insert tabl contents(toc) microsoft ...,['easi way insert tabl content refer microsoft...
3,2019-02-25,107.093452,111.760002,111.260002,112.180000,23750600.0,1.0,2.654672,1.435915e+08,67.938747,...,52185000.0,-15773000.0,-36887000.0,38260000.0,286556000.0,184226000.0,102330000.0,168992000.0,['microsoft introduc help featur microsoft to-...,['microsoft add calendar categori color latest...
4,2019-02-26,107.832413,111.260002,111.169998,113.239998,21536700.0,0.0,2.189771,1.156615e+08,69.433904,...,52185000.0,-15773000.0,-36887000.0,38260000.0,286556000.0,184226000.0,102330000.0,168992000.0,"['build work launch client microsoft, tripism\...","['microsoft, tripism pioneer account-specif su..."


In [11]:
df.tail(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
969,2022-12-22,237.594894,241.259995,233.869995,241.990005,28651700.0,1.0,-20.631162,-4.810033e+08,43.552560,...,89035000.0,-30311000.0,-58876000.0,65149000.0,364840000.0,198298000.0,166542000.0,213574000.0,"[""through jan 2., save popular pc game like fl...",['microsoft store game deals: save 60% elden r...
970,2022-12-23,238.133545,236.110001,233.940002,238.869995,21207000.0,0.0,-19.766353,-2.426366e+08,44.140512,...,89035000.0,-30311000.0,-58876000.0,65149000.0,364840000.0,198298000.0,166542000.0,213574000.0,['microsoft introduc new type meet microsoft t...,['new meet style come microsoft team - uc toda...
971,2022-12-27,236.367981,238.699997,235.830002,238.929993,16688600.0,0.0,-10.456715,-1.358383e+08,42.575151,...,89035000.0,-30311000.0,-58876000.0,65149000.0,364840000.0,198298000.0,166542000.0,213574000.0,['“there’ career cybersecur everyon truli foun...,"['oops, microsoft accident reveal window 12 ne..."
972,2022-12-28,233.944031,236.889999,234.169998,239.720001,17457100.0,1.0,-6.437158,-1.713487e+08,40.454037,...,89035000.0,-30311000.0,-58876000.0,65149000.0,364840000.0,198298000.0,166542000.0,213574000.0,"[""advertisementa decad technolog abund end yea...","['big tech predict 2023: twitter, google, micr..."
973,2022-12-29,240.407837,235.649994,235.649994,241.919998,19770700.0,0.0,-5.346243,-1.260192e+08,47.907113,...,89035000.0,-30311000.0,-58876000.0,65149000.0,364840000.0,198298000.0,166542000.0,213574000.0,['microsoft prepar sever big featur updat wind...,['microsoft plan deliv sever window 11 2023 up...


In [12]:
df.columns

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')

In [13]:
df['y_actual']

0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
969    1.0
970    0.0
971    0.0
972    1.0
973    0.0
Name: y_actual, Length: 974, dtype: float64

In [14]:
y=df['y_actual']
# y=np.array(y)
y

0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
969    1.0
970    0.0
971    0.0
972    1.0
973    0.0
Name: y_actual, Length: 974, dtype: float64

In [15]:
column_names=["y_actual"]

# Create DataFrame by assigning column names
y_df=pd.DataFrame(y, columns=column_names)

In [16]:
y_df

,y_actual
0,1.0
1,1.0
2,1.0
3,1.0
4,0.0
...,...
969,1.0
970,0.0
971,0.0
972,1.0


In [17]:
# headers = ['y_lables']
# y_df = pd.DataFrame(y, columns=headers)
y_df['datetime']=df['datetime']

In [18]:
y_df.head(5)

,y_actual,datetime
0,1.0,2019-02-20
1,1.0,2019-02-21
2,1.0,2019-02-22
3,1.0,2019-02-25
4,0.0,2019-02-26


In [19]:
y_df.shape

(974, 2)

In [20]:
y_df.tail(5)

,y_actual,datetime
969,1.0,2022-12-22
970,0.0,2022-12-23
971,0.0,2022-12-27
972,1.0,2022-12-28
973,0.0,2022-12-29


In [21]:
# df_title=df[['datetime','title']]
df=df.drop(['datetime','title','summary','y_actual'],axis='columns')#dropping the date column


In [22]:
from sklearn.preprocessing import MinMaxScaler
 
 
# scale features
scaler = MinMaxScaler()
 

scaled_data=scaler.fit_transform(df)
 
# print scaled features
df=scaled_data
# print(scaled_data)

In [23]:
df=pd.DataFrame(df)
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000000,0.004038,0.000000,0.000000,0.143354,0.592786,0.712140,0.542931,0.660304,0.569380,...,0.0,0.0,0.0,0.803737,1.0,0.0,0.0,0.07972,0.0,0.0
1,0.009181,0.000000,0.002459,0.006371,0.228053,0.599654,0.736992,0.637608,0.766912,0.583657,...,0.0,0.0,0.0,0.803737,1.0,0.0,0.0,0.07972,0.0,0.0
2,0.015518,0.013251,0.014963,0.013486,0.213284,0.607847,0.752497,0.693349,0.898087,0.602614,...,0.0,0.0,0.0,0.803737,1.0,0.0,0.0,0.07972,0.0,0.0
3,0.018037,0.020444,0.021067,0.017540,0.167698,0.605066,0.735473,0.714294,0.874076,0.615645,...,0.0,0.0,0.0,0.803737,1.0,0.0,0.0,0.07972,0.0,0.0
4,0.021165,0.018341,0.020686,0.021925,0.142547,0.601816,0.729962,0.740025,0.819237,0.625507,...,0.0,0.0,0.0,0.803737,1.0,0.0,0.0,0.07972,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,0.570440,0.565203,0.540799,0.554544,0.223378,0.442286,0.612222,0.294620,0.317915,0.307356,...,1.0,1.0,1.0,0.000000,0.0,1.0,1.0,1.00000,1.0,1.0
970,0.572720,0.543539,0.541096,0.541637,0.138802,0.448332,0.659259,0.304739,0.329265,0.291684,...,1.0,1.0,1.0,0.000000,0.0,1.0,1.0,1.00000,1.0,1.0
971,0.565246,0.554434,0.549108,0.541886,0.087470,0.513411,0.680333,0.277800,0.366969,0.276699,...,1.0,1.0,1.0,0.000000,0.0,1.0,1.0,1.00000,1.0,1.0
972,0.554986,0.546820,0.542071,0.545154,0.096200,0.541510,0.673326,0.241296,0.374807,0.257395,...,1.0,1.0,1.0,0.000000,0.0,1.0,1.0,1.00000,1.0,1.0


In [24]:
df.shape

(974, 25)

In [25]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.6 MB/s eta 0:00:00


In [26]:
# pipe = pipeline("text-classification")
# pipe = pipeline("text-classification", model="roberta-large-mnli")
pipe = pipeline("text-classification", model="roberta-large")

pipe = pipeline("sentiment-analysis")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
titleSentiment=[]

In [28]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [3]:
for content in titleColumn:
           summary=summarizer(content, max_length=512, min_length=60)
           score=pipe(summary)
           
        #    print(summary)
           label=score[0]['label']
           if label=="NEGATIVE":
               titleSentiment.append(0)
           else:
               titleSentiment.append(1)

NameError: ignored

In [ ]:
titleSentiment.count(0)

In [ ]:
titleSentiment.count(1)


In [ ]:
summarySentiment=[]

In [ ]:
for content in summaryColumn:
           score=pipe(content)
           label=score[0]['label']
           if label=="NEGATIVE":
               summarySentiment.append(0)
           else:
               summarySentiment.append(1)

In [ ]:
summarySentiment.count(0)

In [ ]:
summarySentiment.count(1)

In [ ]:
df

In [ ]:

text_vector=[]# will contain vector representation of titleColumn
summary_vector=[]

In [ ]:
for text in titleColumn:
    sentence = text
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    text_vector.append(temp)


In [ ]:
for summary in summaryColumn:
    sentence = summary
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    summary_vector.append(temp)

In [ ]:
text_vector=np.array(text_vector)
summary_vector=np.array(summary_vector)

In [ ]:
text_vector.shape

In [ ]:
type(text_vector)

In [ ]:
summary_vector.shape

In [ ]:
text_vector.shape

In [ ]:
scaler = MinMaxScaler()
scaled_data_text=scaler.fit_transform(text_vector)
# scaled_data_text=model.transform(text_vector)
 
# print scaled features
# print(scaled_data)

In [ ]:
scaler = MinMaxScaler()
scaled_data_summary=scaler.fit_transform(summary_vector)
# scaled_data_summary=model.transform(summary_vector)
 
# print scaled features
# print(scaled_data)

In [ ]:
df

In [ ]:
pca = PCA(n_components = 30)
pca.fit(scaled_data_text)
data_pca_text = pca.transform(scaled_data_text)
data_pca_text

In [ ]:
pca = PCA(n_components = 30)
pca.fit(scaled_data_summary)
data_pca_summary = pca.transform(scaled_data_summary)
data_pca_summary

In [ ]:
data_pca_text.shape

In [ ]:
data_pca_summary.shape

In [ ]:
df

In [ ]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_text)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10','pca11','pca12','pca13','pca14','pca15','pca16','pca17','pca18','pca19','pca20','pca21','pca22','pca23','pca24','pca25','pca26','pca27','pca28','pca29','pca30']
df_news_text=pd.DataFrame(data_pca_text, columns = mycolumns)

In [ ]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_summary)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['Spca1','Spca2','Spca3','Spca4','Spca5','Spca6','Spca7','Spca8','Spca9','Spca10','Spca11','Spca12','Spca13','Spca14','Spca15','Spca16','Spca17','Spca18','Spca19','Spca20','Spca21','Spca22','Spca23','Spca24','Spca25','Spca26','Spca27','Spca28','Spca29','Spca30']
df_news_summary=pd.DataFrame(data_pca_summary, columns = mycolumns)

In [ ]:
df_news_text

In [ ]:
df_news_summary

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df=np.array(df)

In [ ]:
column=['close', 'open', 'low', 'high', 'volume','evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
  'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
   'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization']

df=pd.DataFrame(df,columns=column)
    #  'volume', 'y_actual', 'evm',
#    ,'titleSentiment','summarySentiment'

In [ ]:
df

In [ ]:
df['titleSentiment']=titleSentiment

In [ ]:
df['summarySentiment']=summarySentiment

In [ ]:
# df=df.drop(['y_actual'],axis='columns')
df.columns

In [ ]:
df.head(5)

In [ ]:
newDF=pd.concat([dateColumn,df,df_news_text,df_news_summary],axis=1)


In [ ]:
newDF.shape

In [ ]:
newDF.head(5)

In [ ]:
newDF.columns

Start applying model after this.

Take data before 2022-9-30 as training and after as testing.


In [ ]:
file_train = newDF[(newDF['datetime'] <'2022-01-01')]
file_test = newDF[(newDF['datetime'] >='2022-01-01')]
y_df_train = y_df[(y_df['datetime'] <'2022-01-01')]
y_df_test = y_df[(y_df['datetime'] >='2022-01-01')]
print("train x: ",file_train.shape,"train y: ",y_df_train.shape)
print("test x: ",file_test.shape,"test y: ",y_df_test.shape)



In [ ]:
file_train = file_train.drop('datetime', axis=1)
# file_train = file_train.drop('title', axis=1)
file_test = file_test.drop('datetime', axis=1)
y_df_train = y_df_train.drop('datetime', axis=1)
y_df_test = y_df_test.drop('datetime', axis=1)
file_train

In [ ]:
file_test

In [ ]:
y_df_train

In [ ]:
y_df_test

In [ ]:
print(y_df_test)

In [ ]:
type(file_train)
file_train = file_train.to_numpy()

In [ ]:
type(file_test)
file_test = file_test.to_numpy()

In [ ]:
type(y_df_train)
y_df_train = y_df_train.to_numpy()

In [ ]:
type(y_df_test)
y_df_test = y_df_test.to_numpy()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report




baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
baseline_LR.fit(file_train, y_df_train)
    # print('Coefficients: \n', baseline_LR.coef_)
print("training accracy: %.4f" % baseline_LR.score(file_train, y_df_train))
print("test accuracy: %.4f" % baseline_LR.score(file_test, y_df_test))
pred= baseline_LR.predict(file_train)
y_fit_LR = baseline_LR.predict(file_test)
print(type(y_fit_LR))
# print(y_fit_LR)
# print(y_df_test)
print('Accuracy:', accuracy_score(y_df_test, y_fit_LR))
print('ROC AUC Score:', roc_auc_score(y_df_test, y_fit_LR))
print('F1 score:', f1_score(y_df_test, y_fit_LR))
print()
print('\n clasification report training:\n', classification_report(y_df_train, pred))
print('\n confussion matrix:\n',confusion_matrix(y_df_train, pred))

print('\n clasification report testing:\n', classification_report(y_df_test, y_fit_LR))
print('\n confussion matrix:\n',confusion_matrix(y_df_test, y_fit_LR))

In [ ]:
list1=y_df_test.tolist()
plt.plot(list1,linestyle = '--')
plt.show()

In [ ]:
list2=y_fit_LR.tolist()
plt.plot(list2,linestyle = '--')
plt.show()

In [ ]:
# print(y_df_test)
type(y_df_test)


In [ ]:
y_df_test.shape

In [ ]:
print(y_fit_LR)
print(type(y_fit_LR))


In [ ]:
y_fit_LR.shape

In [ ]:
#Predicted vs True Adj Close Value – LR



plt.plot(y_df_test, label='True Value')
plt.plot(y_fit_LR, label='LR Value')
plt.title('Prediction by Logistic Regression')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()

In [ ]:
# Fit and report Logistic Regression
baseline_LR = LogisticRegression(C = 0.000001, tol=0.000000001, max_iter=1000)
baseline_LR.fit(file_train, y_df_train)
    # print('Coefficients: \n', baseline_LR.coef_)
print("training accracy: %.4f" % baseline_LR.score(file_train, y_df_train))
print("test accuracy: %.4f" % baseline_LR.score(file_test, y_df_test))
pred= baseline_LR.predict(file_train)
y_fit_LR = baseline_LR.predict(file_test)
print(type(y_fit_LR))
# print(y_fit_LR)
# print(y_df_test)
print('Accuracy:', accuracy_score(y_df_test, y_fit_LR))
print('ROC AUC Score:', roc_auc_score(y_df_test, y_fit_LR))
print('F1 score:', f1_score(y_df_test, y_fit_LR))
print()
print('\n clasification report training:\n', classification_report(y_df_train, pred))
print('\n confussion matrix:\n',confusion_matrix(y_df_train, pred))

print('\n clasification report testing:\n', classification_report(y_df_test, y_fit_LR))
print('\n confussion matrix:\n',confusion_matrix(y_df_test, y_fit_LR))

In [ ]:
from sklearn import svm

In [ ]:
# Fit and report SVM
    # c = 0.5
svm_rbf = svm.SVC(C=0.5, kernel='rbf',max_iter=50000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

In [ ]:
# Fit and report SVM
    # c = 1e10
svm_rbf = svm.SVC(C=1e10, kernel='rbf',max_iter=100000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

In [ ]:
# Fit and report SVM
    # c = 1e1
svm_rbf = svm.SVC(C=10, kernel='rbf',max_iter=100000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Fit Random Forest
rf = RandomForestClassifier(n_estimators=10, oob_score=True, random_state=0)
rf.fit(file_train, y_df_train)
# Training acc
pred = rf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Random Forest is ', accuracy)
# Test acc
y_hat = rf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Random Forest is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

In [ ]:
# Fit Random Forest
rf = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=0)
rf.fit(file_train, y_df_train)
# Training acc
pred = rf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Random Forest is ', accuracy)
# Test acc
y_hat = rf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Random Forest is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop,Adam
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [ ]:
m1, n1 = file_train.shape
x_train = np.array(file_train).reshape(m1, n1, 1)
m2, n2 = file_test.shape
x_test= np.array(file_test).reshape(m2, n2, 1)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
train_confusion = np.zeros((2, 2))
test_confusion = np.zeros((2, 2))

batch_size = 128
epochs = 80
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
print(x_train.shape)
# print(filename)
# if x_train.shape[0] == 0:
#     continue

# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             

# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

In [ ]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

batch_size = 32
epochs = 200
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
print(x_train.shape)


# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             
# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

In [ ]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

# batch_size = 16
# epochs = 500
# model = Sequential()
# model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

# #model.add(Conv1D(128, 3, activation='relu'))
# model.add(MaxPooling1D(3))

# #model.add(Conv1D(128, 3, activation='relu'))
# model.add(Conv1D(128, 3, activation='relu'))
# model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='sigmoid'))
# """
# #model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# """
# model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# print(x_train.shape)


# # Train model
# history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                


batch_size = 64
epochs = 200
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

# model.add(Conv1D(256, 3, activation='relu'))
# model.add(MaxPooling1D(3))

model.add(Conv1D(512, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.9))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.9))

model.add(Dense(1, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# print(x_train.shape)

# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size, epochs=epochs,verbose=1)


    # Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [ ]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=250, learning_rate=1)# 200 0.01
                        
# Train Adaboost Classifer
model = abc.fit(file_train, y_df_train)

pred = model.predict(file_train)

#Predict the response for test dataset
y_pred = model.predict(file_test)

# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:",metrics.accuracy_score(y_df_train, pred))
print("Testing Accuracy:",metrics.accuracy_score(y_df_test, y_pred))


print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_pred))




In [ ]:
# Load libraries


# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation

svc=SVC(probability=True, kernel='rbf')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=250, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(file_train, y_df_train)

pred = model.predict(file_train)

#Predict the response for test dataset
y_pred = model.predict(file_test)


# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:",metrics.accuracy_score(y_df_train, pred))

print("Testing Accuracy:",metrics.accuracy_score(y_df_test, y_pred))




print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_pred))

